In [11]:
import pandas as pd

In [25]:
data = pd.read_csv('../GSAF5.csv', engine="python")
print(data.shape)

data.columns


(5992, 24)


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

DATA CLEANING

In [28]:
#1 MISSING VALUES
data

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [169]:
data.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number.1             object
Case Number.2             object
original order             int64
Unnamed: 22               object
Unnamed: 23               object
dtype: object

In [73]:
#CHECKING IF THERE ARE DUPLICATED COLUMNS, AS IT SEEMS SO FROM FIRST LOOK
def getDuplicateColumns(df):  
    duplicateColumnNames = []
    for x in range(df.shape[1]):
        col = df.iloc[:, x]
        for y in range(x + 1, df.shape[1]):
            otherCol = df.iloc[:, y]
            if col.equals(otherCol):
                duplicateColumnNames.append(df.columns.values[y])
            else:
                pass
 
    return duplicateColumnNames

getDuplicateColumns(data)

[]

In [97]:
#data.duplicated(['href', 'href formula'])
#df2 = data.loc[:, ['href', 'href formula']]

#if df2['href'].equals['href formula'].value_count():
    return 
    
        

TypeError: 'method' object is not subscriptable

In [72]:
data.duplicated(['href', 'href formula']).value_counts()


False    5981
True       11
dtype: int64

In [74]:
data.duplicated(['Case Number.1', 'Case Number.2']).value_counts()

False    5976
True       16
dtype: int64

In [76]:
data.duplicated(['Case Number', 'Year']).value_counts()

False    5977
True       15
dtype: int64

In [85]:
duplicates=[]
for i in data['href']:
    for e in data['href formula']:
        if i is e:
            duplicates.append(i)
print(duplicates)

[nan, nan, nan]


In [118]:
data

Cols = list(data.columns)
for i,item in enumerate(data.columns):
    if item in data.columns[:i]: Cols[i] = "toDROP"
data.columns = Cols
print(cols)
#df = df.drop("toDROP",1)


#data = data.loc[:,~data.columns.duplicated()]
#display(data.head())
#print(data.shape)

NameError: name 'cols' is not defined

In [121]:
#Case Number repeated, keeping only 1 column, checking that other two were dropped
data.loc[:, ['Case Number','Case Number.1','Case Number.2']]

data1 = data.drop(['Case Number.1', 'Case Number.2'], axis=1)
print(data.shape)
print(data1.shape)

(5992, 24)
(5992, 22)


In [124]:
#Href and pdf repeated, keeping only 1 column, checking that other two were dropped
data1.loc[:, ['pdf','href','href formula']]

data2 = data1.drop(['pdf', 'href formula'], axis=1)
print(data1.shape)
print(data2.shape)









(5992, 22)
(5992, 20)


In [151]:
#Delete Useless Columns from a qialitative point of view: Name, Investigator
data3 = data2.drop(['Name', 'Investigator or Source'], axis=1)
print(data2.shape)
print(data3.shape)




(5992, 20)
(5992, 18)


In [127]:
#check for NULL values in columns 
print(data3.shape)
data3.isnull().sum()

(5992, 18)


Case Number          0
Date                 0
Year                 0
Type                 0
Country             43
Area               402
Location           496
Activity           527
Sex                567
Age               2681
Injury              27
Fatal (Y/N)         19
Time              3213
Species           2934
href                 3
original order       0
Unnamed: 22       5991
Unnamed: 23       5990
dtype: int64

In [155]:
#Drop columns with more than 40% Nulls: Age, Time, Species, Unnamed:22 and Unnamed 23
cutoff_null = .6 * len(data3)
data3.dropna(thresh = cutoff_null, axis = 1, inplace = True)
data3.shape

#columns_todrop = data3.columns
#data4 = data3.drop(['Age', 'Time', 'Species', 'Unnamed: 22', 'Unnamed: 23'], axis=1)
#print(data3.shape)
#print(data4.shape)

(5992, 13)

In [157]:
data3


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal (Y/N),href,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,Minor injury to thigh,N,http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,Lacerations to hands,N,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,Lacerations to lower leg,N,http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,Struck by fin on chest & leg,N,http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,No injury: Knocked off board by shark,N,http://sharkattackfile.net/spreadsheets/pdf_di...,5989
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,NaN,Shark rammed boat. No injury to occupant,N,http://sharkattackfile.net/spreadsheets/pdf_di...,5988
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,M,Minor injury to arm,N,http://sharkattackfile.net/spreadsheets/pdf_di...,5987
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,F,Severe lacerations to shoulder & forearm,N,http://sharkattackfile.net/spreadsheets/pdf_di...,5986
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,M,FATAL,Y,http://sharkattackfile.net/spreadsheets/pdf_di...,5985
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,F,Lacerations & punctures to lower right leg,N,http://sharkattackfile.net/spreadsheets/pdf_di...,5984


In [159]:
#drop row if year below 1900, only interested in the past 2 centuries
data4 = data3[data3.Year >1900]
data4.shape

(5311, 13)

In [170]:
#Look at data types to see what numeric values are not appearing as numeric
data4.dtypes

Case Number       object
Date              object
Year               int64
Type              object
Country           object
Area              object
Location          object
Activity          object
Sex               object
Injury            object
Fatal (Y/N)       object
href              object
original order     int64
dtype: object

In [173]:
data4.describe()

,Year,original order
count,5311.000000,5311.000000
mean,1977.481642,3337.867445
std,30.858196,1533.517125
min,1901.000000,682.000000
25%,1957.000000,2010.500000
50%,1984.000000,3338.000000
75%,2005.000000,4665.500000
max,2016.000000,5993.000000


In [174]:
data4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5311 entries, 0 to 5311
Data columns (total 13 columns):
Case Number       5311 non-null object
Date              5311 non-null object
Year              5311 non-null int64
Type              5311 non-null object
Country           5289 non-null object
Area              5016 non-null object
Location          4948 non-null object
Activity          4886 non-null object
Sex               4807 non-null object
Injury            5292 non-null object
Fatal (Y/N)       5302 non-null object
href              5309 non-null object
original order    5311 non-null int64
dtypes: int64(2), object(11)
memory usage: 580.9+ KB


In [ ]:
#Eliminating Duplicates
#data4 = data4.drop_diplicates()
#print before - after len data

In [167]:
# Cleaning Date Column from characters etc, do this for all columns
import re
data4['Date'] = data4['Date'].str.replace('Reported', '')
data4['Date'] = data4['Date'].str.replace('\(´`', '')

data4['Date'] = data4['Date'].str.replace('Summer', 'Jul')
data4['Date'] = data4['Date'].str.replace('Mid', '')

data4['Date']

/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is 

0           18-Sep-16
1           18-Sep-16
2           18-Sep-16
3           17-Sep-16
4           16-Sep-16
5           15-Sep-16
6           11-Sep-16
7           07-Sep-16
8           06-Sep-16
9           05-Sep-16
10          05-Sep-16
11          04-Sep-16
12          01-Sep-16
13          29-Aug-16
14          29-Aug-16
15          27-Aug-16
16          25-Aug-16
17          07-Aug-16
18          06-Aug-16
19          04-Aug-16
20          29-Jul-16
21          28-Jul-16
22          28-Jul-16
23          27-Jul-16
24          26-Jul-16
25          24-Jul-16
26          23-Jul-16
27          23-Jul-16
28          20-Jul-16
29          17-Jul-16
            ...      
5282        04-May-03
5283      20-Mar-1903
5284        12-Mar-03
5285      10-Jan-1903
5286      Jul of 1903
5287         Ca. 1903
5288      22-Dec-1902
5289        10-Nov-02
5290      01-Nov-1902
5291      29-Aug-1902
5292      24-Aug-1902
5293        08-Aug-02
5294        15-Jul-02
5295        06-Jul-02
5296      

In [ ]:
#Cambiar Column Order

In [ ]:
#Preffer to analyse area than country as country too big, list major areas and countries